# CNN

Baseado no código disponibilizado pelo professor Minetto. A arquitetura da rede foi sendo modificada em busca de uma melhor acurácia. Funções para minimização de erro e funções de custo também foram trocadas, observando-se o resultado. O número de épocas e a taxa de aprendizado também foram testados. 

In [1]:
import tensorflow as tf
import numpy as np
import cv2
from tqdm import tqdm
from random import shuffle
import os
from sklearn.preprocessing import StandardScaler  

C:\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:472: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [2]:
#---------------------------------------------
#Settings: definicao dos parametros da rede:
n_classes = 2                     # A base de dados DogsAndCats tem 2 classes de objetos!!!!
nepochs = 10                      # Numero de epocas para o treinamento!!!
batch_size = 32                   # Numero de imagens por batch!!!
image_size = 32                   # Todas as imagens devem ser redimensionadas para 32x32 pixels!!!
nchannels = 3                     # Numero de canais de cores na imagem!!!
n_input = image_size * image_size * nchannels # Tamanho da entrada!
learning_rate = 1e-3              # Taxa de aprendizado!!!
kprob = 0.5                       # Probabilidade para dropout!!!
TRAIN_DIR = 'kaggle/train'
TEST_DIR = 'kaggle/test'

#---------------------------------------------
def convolutional_neural_network (x, prob):
    
    input_layer = tf.reshape(x, shape=[-1, image_size, image_size, nchannels])
    
    #Primeira camada de convolucao:
    conv1 = tf.layers.conv2d (
                inputs=input_layer,
                filters=32,
                kernel_size=[3, 3],
                padding="same",
                activation=tf.nn.relu
            )
    
    #Primeira camada de pooling:
    pool1 = tf.layers.max_pooling2d (
                inputs=conv1, 
                pool_size=[2, 2], 
                strides=2
            )
    
    #Segunda camada de convolucao:
    conv2 = tf.layers.conv2d (
                inputs=pool1,
                filters=64,
                kernel_size=[3, 3],
                padding="same",
                activation=tf.nn.relu
            )
    
    #Segunda camada de pooling:
    pool2 = tf.layers.max_pooling2d (
                inputs=conv2, 
                pool_size=[2, 2], 
                strides=2
            )
    
    #Terceira camada de convolucao:
    conv3 = tf.layers.conv2d (
                inputs=pool2,
                filters=128,
                kernel_size=[3, 3],
                padding="same",
                activation=tf.nn.relu
            )
    
    #Terceira camada de pooling:
    pool3 = tf.layers.max_pooling2d (
                inputs=conv3, 
                pool_size=[2, 2], 
                strides=2
            )
    
    #Quarta camada de convolucao:
    conv4 = tf.layers.conv2d (
                inputs=pool3,
                filters=256,
                kernel_size=[3, 3],
                padding="same",
                activation=tf.nn.relu
            )
    
    #Quarta camada de pooling:
    pool4 = tf.layers.max_pooling2d (
                inputs=conv4, 
                pool_size=[2, 2], 
                strides=2
            )

    
  
    flat = tf.contrib.layers.flatten (pool4)

    fc1 = tf.contrib.layers.fully_connected (inputs=flat, num_outputs=16, activation_fn=tf.nn.relu)
  
    fc1 = tf.nn.dropout (fc1, prob)
  
    out = tf.contrib.layers.fully_connected(inputs=fc1, num_outputs=n_classes, activation_fn=None)

    return out

#---------------------------------------------
def create_label(image_name):
    word_label = image_name.split('.')[-3]
    if word_label == 'cat':
        return np.array([1,0])
    elif word_label == 'dog':
        return np.array([0,1])
    else: 
        print ("Esta classe não existe!!!!!")

#---------------------------------------------
def read_dataset (filename):
    X = []
    Y = []
    for img in tqdm(os.listdir(filename)):
        path = os.path.join(filename, img)
        img_data = cv2.imread(path)
        img_data = cv2.resize(img_data, (image_size, image_size)) #deixa com as dimensões definidas
        
        #normalização
        img_data = img_data/255.0    
        
        #cria os vetores de dados e de labels
        X.append(np.array(img_data))
        Y.append(np.array(create_label(img)))
        
    return X,Y

#---------------------------------------------
def next_batch (num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = data[idx]
    labels_shuffle = labels[idx]
    return data_shuffle, labels_shuffle

#---------------------------------------------
if __name__ == "__main__":

    
    # Leitura da base de dados:
    X_train,Y_train = read_dataset (TRAIN_DIR)
    X_train = np.asarray(X_train).reshape(-1, n_input)
    Y_train = np.asarray(Y_train)

    X_test,Y_test = read_dataset (TEST_DIR)
    X_test = np.asarray(X_test).reshape(-1, n_input)
    Y_test = np.asarray(Y_test)
    

    # Variáveis do tensorflow:
    Y = tf.placeholder(tf.float32, [None, n_classes])
    X = tf.placeholder(tf.float32, [None, image_size * image_size * nchannels])
    prob = tf.placeholder(tf.float32, name='keep_prob')

    Ypred = convolutional_neural_network (X, kprob)

    # Funções de custo:
    error1 = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(tf.nn.softmax(Ypred)), reduction_indices = [1]))
    error2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Ypred, labels = Y)) 
    error3 = tf.reduce_mean(tf.reduce_sum(tf.square(tf.nn.softmax(Ypred) - Y), reduction_indices = [1]))
    error = error2
    
    # Funções para minimização de erro: 
    optimizer1 = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(error)
    optimizer2 = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(error)
    optimizer = optimizer2
 
    corr = tf.equal(tf.argmax(Ypred,1),tf.argmax(Y,1))
 
    accuracy = tf.reduce_mean(tf.cast(corr,tf.float32))

    # Inicialização de variáveis:
    init = tf.initialize_all_variables()

    with tf.Session() as sess:
        sess.run(init)

        # Treino:
        for epoch in range(nepochs):
            train_err = 0
            train_acc = 0
            train_batches = 0
            total_batch = int(len(X_train)/batch_size)
            for i in range(total_batch):
                batch_xs, batch_ys = next_batch (batch_size, X_train, Y_train)
                sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys})
                err, acc = sess.run([error,accuracy], feed_dict={X: batch_xs, Y: batch_ys, prob: kprob})
                train_err += err
                train_acc += acc
                train_batches += 1
            print("Epoch: ", '%2d' % (epoch+1))
            print("  training loss:\t\t{:.6f}".format(train_err/train_batches))
            print("  validation accuracy:\t\t{:.2f} %".format(train_acc/train_batches * 100))

        # Testes:
        test_err = 0
        test_acc = 0
        test_batches = 0
        total_batch = int(len(X_test)/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = next_batch (batch_size, X_test, Y_test)
            sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys})
            err, acc = sess.run([error,accuracy], feed_dict={X: batch_xs, Y: batch_ys, prob: kprob})
            test_err += err
            test_acc += acc
            test_batches += 1
        print("Final results:")
        print("  test loss:\t\t\t{:.6f}".format(test_err/test_batches))
        print("  test accuracy:\t\t{:.2f} %".format((test_acc/test_batches)*100))

100%|██████████████████████████████████████████████████████████████████████████████| 4444/4444 [03:19<00:00, 22.27it/s]


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch:   1
  training loss:		0.671400
  validation accuracy:		57.35 %
Epoch:   2
  training loss:		0.567082
  validation accuracy:		70.34 %
Epoch:   3
  training loss:		0.490612
  validation accuracy:		76.91 %
Epoch:   4
  training loss:		0.446322
  validation accuracy:		79.60 %
Epoch:   5
  training loss:		0.406091
  validation accuracy:		81.57 %
Epoch:   6
  training loss:		0.376110
  validation accuracy:		83.25 %
Epoch:   7
  training loss:		0.328262
  validation accuracy:		86.31 %
Epoch:   8
  training loss:		0.287737
  validation accuracy:		88.33 %
Epoch:   9
  training loss:		0.264059
  validation accuracy:		89.38 %
Epoch:  10
  training loss:		0.233496
  validation accuracy:		91.00 %
Final results:
  test loss:			0.429837
  test accuracy:		80.59 %
